# 01 - Classificação com Validação Cruzada Estratificada

Este notebook realiza o pré-processamento, separação de atributos/rótulos e avaliação de três algoritmos supervisionados:
- Árvore de Decisão (Random Forest)
- K-NN (K-Nearest Neighbors)
- MLP (Multi-Layer Perceptron)

A avaliação utiliza F1-score e matriz de confusão via validação cruzada estratificada k-fold, com `random_state` fixo para reprodutibilidade.

Observação: Ajuste o caminho do arquivo da base de dados na seção de carregamento.


In [1]:
import pandas as pd

# Configurações gerais
RANDOM_STATE = 42
N_JOBS = -1
N_SPLITS = 5  # k-fold
TEST_SIZE = 0.2

# Exibição de opções do pandas
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 120)


In [15]:
# Imports
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import (
    StratifiedKFold, 
    cross_validate, 
    cross_val_predict, 
    cross_val_score,
    train_test_split
)
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.metrics import f1_score, confusion_matrix, ConfusionMatrixDisplay, make_scorer

from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from tensorflow.keras.utils import to_categorical


## Carregamento de dados

Substitua `data/arquivo.csv` pelo caminho real. O rótulo (target) deve estar em uma coluna, por exemplo `target`. Se a base tiver valores ausentes, faremos tratamento na etapa de pré-processamento.


## Pré-processamento

- Identificação de colunas numéricas e categóricas
- Tratamento de valores ausentes
- Padronização de numéricas e One-Hot em categóricas
- Separação de `X` e `y`


In [3]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [4]:
# Carregando o arquivo JSON
sinal = pd.read_json('../data/Sinais/Sinais/Adição_AP_1.json')

# Extraindo todos os frames
frames = sinal['frames']

# Criando uma lista para armazenar os dados de todos os keypoints
dados_keypoints = []

# Iterando por todos os frames
for frame_idx, frame_data in enumerate(frames):
    # Iterando por todos os keypoints no frame
    for keypoint in frame_data['keypoints']:
        # Adicionando os dados do keypoint à lista
        dados_keypoints.append({
            'frame': frame_idx,
            'id': keypoint['id'],
            'x': keypoint['x'],
            'y': keypoint['y'],
            'z': keypoint['z'],
            'visibility': keypoint['visibility']
        })

# Convertendo para DataFrame
df_keypoints = pd.DataFrame(dados_keypoints)

# Calculando métricas para cada ID e cada atributo (x, y, z, visibility)
metricas = df_keypoints.groupby('id').agg({
    'x': ['mean', 'std', 'min', 'max'],
    'y': ['mean', 'std', 'min', 'max'],
    'z': ['mean', 'std', 'min', 'max'],
    'visibility': ['mean', 'std', 'min', 'max']
})

print("Métricas calculadas para cada ID:")
display(metricas)

Métricas calculadas para cada ID:


x                                 y                                 z                       visibility  \
          mean        std  min  max         mean         std   min   max      mean       std   min   max       mean   
id                                                                                                                    
0   442.692308   6.140641  436  454   187.951049   11.869792   178   212 -1.271958  0.187734 -1.52 -0.89   1.000000   
1   467.713287   7.276842  461  481   156.468531    9.763978   147   176 -1.199790  0.178003 -1.43 -0.84   1.000000   
2   481.048951   6.061107  474  492   157.384615   10.097359   148   177 -1.200140  0.178278 -1.43 -0.84   1.000000   
3   491.580420   5.785378  484  502   158.566434   10.311049   149   179 -1.199860  0.178278 -1.43 -0.84   1.000000   
4   425.160839   7.259428  418  439   155.608392    8.369154   147   171 -1.197622  0.186966 -1.44 -0.82   1.000000   
5   411.412587   6.756665  405  424   156.055944    7.828061   147   170 -1.197972  0.187172 -1.44 -0.82   1.000000   
6   398.300699   7.045161  391  411   156.930070    7.223446   148   170 -1.198951  0.187432 -1.44 -0.82   1.000000   
7   509.797203   8.184113  502  524   176.216783    9.457198   166   194 -0.701678  0.153053 -0.89 -0.40   1.000000   
8   384.797203   4.798121  380  395   175.104895    6.603150   167   188 -0.681049  0.190140 -0.88 -0.31   1.000000   
9   471.244755   4.837088  465  479   226.846154   10.973276   219   249 -1.082098  0.177406 -1.31 -0.73   1.000000   
10  417.741259   5.932381  410  427   226.349650   10.133099   218   247 -1.079021  0.187069 -1.31 -0.71   1.000000   
11  625.783217   2.368038  622  630   382.146853    3.339811   375   390 -0.429720  0.069919 -0.58 -0.30   1.000000   
12  278.272727   5.061845  271  290   384.139860   10.303562   360   397 -0.354685  0.160925 -0.54 -0.04   1.000000   
13  697.811189  15.834510  666  724   641.321678    5.070769   632   653 -0.563846  0.244736 -1.07 -0.31   0.998951   
14  164.559441  71.486099   46  225   606.454545   78.719034   462   664 -0.558322  0.205299 -1.01 -0.31   1.000000   
15  530.076923  39.007778  460  563   712.734266  134.443818   462   807 -1.189371  0.304604 -1.97 -0.79   0.986783   
16  335.258741  60.358986  224  386   672.517483  197.264920   332   826 -1.239161  0.374545 -2.19 -0.85   0.980280   
17  489.965035  43.443084  417  528   750.202797  179.724616   419   878 -1.370070  0.299113 -2.17 -0.93   0.960070   
18  392.335664  60.265720  280  447   713.671329  230.283193   313   892 -1.405594  0.382353 -2.41 -1.01   0.943357   
19  476.937063  35.930531  417  512   719.734266  177.897856   393   844 -1.381469  0.238005 -2.07 -0.91   0.962168   
20  420.111888  57.379648  315  469   684.237762  215.689101   309   855 -1.412797  0.303965 -2.34 -1.08   0.945245   
21  483.916084  33.788175  428  517   703.769231  163.030313   404   821 -1.201049  0.280635 -1.95 -0.78   0.961469   
22  409.328671  56.236645  305  459   673.846154  202.098125   325   835 -1.251888  0.352460 -2.18 -0.88   0.943147   
23  565.104895   2.827713  554  570   901.468531    6.165333   889   912 -0.045944  0.017532 -0.08 -0.02   0.978042   
24  340.678322   6.666715  328  351   906.251748    4.317453   889   915  0.048112  0.017237  0.02  0.08   0.977483   
25  579.097902   8.409681  553  590  1274.902098   13.393984  1232  1299 -0.093427  0.140327 -0.31  0.18   0.200629   
26  356.314685  22.079377  317  377  1273.531469   11.935156  1234  1292  0.246713  0.114251 -0.05  0.49   0.136294   
27  585.307692  15.560980  555  603  1599.377622   25.511887  1509  1637  0.836853  0.136879  0.56  1.16   0.010909   
28  378.860140  28.155654  330  405  1604.391608   21.567462  1520  1633  1.055804  0.133582  0.73  1.38   0.016923   
29  590.041958  13.917703  559  608  1645.048951   29.095116  1551  1691  0.900979  0.138642  0.62  1.23   0.010629   
30  378.860140  30.525123  327  407  1654.335664   21.812241  1567  1683  1.121748  0.

### Montando o dataframe

As partes com menor visibilidade e que no geral não são tão importantes para libras serão retiradas do df.

A visibilidade a partir do keypoint 25 (dos Joelhos para baixo) é muito baixa e também não é tão importante na língua de sinais.

In [16]:
sinais = pd.read_csv('../data/Sinais/sinais.csv')
print(sinais.shape)
sinais.head()

def extrair_metricas_do_arquivo(caminho_arquivo):
    # Carregando o arquivo JSON
    sinal = pd.read_json(caminho_arquivo)
    
    # Extraindo todos os frames
    frames = sinal['frames']
    
    # Criando uma lista para armazenar os dados de todos os keypoints
    dados_keypoints = []
    
    # Iterando por todos os frames
    for frame_idx, frame_data in enumerate(frames):
        # Iterando por todos os keypoints no frame
        for keypoint in frame_data['keypoints']:
            # Adicionando apenas os keypoints de 0 a 25
            if keypoint['id'] <= 24:
                dados_keypoints.append({
                    'frame': frame_idx,
                    'id': keypoint['id'],
                    'x': keypoint['x'],
                    'y': keypoint['y'],
                    'z': keypoint['z'],
                    'visibility': keypoint['visibility']
                })
    
    # Convertendo para DataFrame
    df_keypoints = pd.DataFrame(dados_keypoints)
    
    # Calculando métricas para cada ID e cada atributo (x, y, z, visibility)
    metricas = df_keypoints.groupby('id').agg({
        'x': ['mean', 'std', 'min', 'max'],
        'y': ['mean', 'std', 'min', 'max'],
        'z': ['mean', 'std', 'min', 'max'],
        'visibility': ['mean', 'std', 'min', 'max']
    })
    
    # Achatando o índice de colunas e renomeando
    metricas.columns = ['_'.join(col).strip() for col in metricas.columns.values]
    
    # Resetando o índice para transformar o ID em coluna
    metricas = metricas.reset_index()
    
    # Criando um dicionário com todas as métricas
    metricas_dict = {}
    for _, row in metricas.iterrows():
        id_keypoint = int(row['id'])  # Convertendo para inteiro
        for col in metricas.columns:
            if col != 'id':
                metricas_dict[f'id_{id_keypoint}_{col}'] = row[col]
    
    return metricas_dict

# Criando um DataFrame para armazenar os resultados
resultados = pd.DataFrame()

# Processando cada arquivo
for idx, row in sinais.iterrows():
    # Construindo o caminho completo para o arquivo JSON
    caminho_arquivo = f'../data/Sinais/Sinais/{row["file_name"]}'
    
    try:
        # Extraindo métricas do arquivo
        metricas_arquivo = extrair_metricas_do_arquivo(caminho_arquivo)
        
        # Criando um DataFrame temporário com os dados originais e as métricas
        temp_df = pd.DataFrame([{**row, **metricas_arquivo}])
        
        # Concatenando com o DataFrame de resultados
        resultados = pd.concat([resultados, temp_df], ignore_index=True)
        
        # Feedback para acompanhar o progresso
        if idx % 10 == 0:
            print(f"Processados {idx+1} arquivos de {len(sinais)}")
            
    except Exception as e:
        print(f"Erro ao processar o arquivo {row['file_name']}: {e}")


(2502, 7)
Processados 1 arquivos de 2502
Processados 11 arquivos de 2502
Processados 21 arquivos de 2502
Processados 31 arquivos de 2502
Processados 41 arquivos de 2502
Processados 51 arquivos de 2502
Processados 61 arquivos de 2502
Processados 71 arquivos de 2502
Processados 81 arquivos de 2502
Processados 91 arquivos de 2502
Processados 101 arquivos de 2502
Processados 111 arquivos de 2502
Processados 121 arquivos de 2502
Processados 131 arquivos de 2502
Processados 141 arquivos de 2502
Processados 151 arquivos de 2502
Processados 161 arquivos de 2502
Processados 171 arquivos de 2502
Processados 181 arquivos de 2502
Processados 191 arquivos de 2502
Processados 201 arquivos de 2502
Processados 211 arquivos de 2502
Processados 221 arquivos de 2502
Processados 231 arquivos de 2502
Processados 241 arquivos de 2502
Processados 251 arquivos de 2502
Processados 261 arquivos de 2502
Processados 271 arquivos de 2502
Processados 281 arquivos de 2502
Processados 291 arquivos de 2502
Processados

/var/folders/ns/qqkzf_x52rqdpshm0vpl205r0000gn/T/ipykernel_9168/4130230414.py:7: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  sinal = pd.read_json(caminho_arquivo)


Processados 571 arquivos de 2502
Processados 581 arquivos de 2502
Processados 591 arquivos de 2502
Processados 601 arquivos de 2502
Processados 611 arquivos de 2502
Processados 621 arquivos de 2502
Processados 631 arquivos de 2502
Processados 641 arquivos de 2502
Processados 651 arquivos de 2502
Processados 661 arquivos de 2502
Processados 671 arquivos de 2502
Processados 681 arquivos de 2502
Processados 691 arquivos de 2502
Processados 701 arquivos de 2502
Processados 711 arquivos de 2502
Processados 721 arquivos de 2502
Processados 731 arquivos de 2502
Processados 741 arquivos de 2502
Processados 751 arquivos de 2502
Processados 761 arquivos de 2502
Processados 771 arquivos de 2502
Processados 781 arquivos de 2502
Processados 791 arquivos de 2502
Processados 801 arquivos de 2502
Processados 811 arquivos de 2502
Processados 821 arquivos de 2502
Processados 831 arquivos de 2502
Processados 841 arquivos de 2502
Processados 851 arquivos de 2502
Processados 861 arquivos de 2502
Processado

In [17]:
resultados.head()


,file_name,width,height,duration_sec,num_frames,sinal,interprete,id_0_x_mean,id_0_x_std,id_0_x_min,id_0_x_max,id_0_y_mean,id_0_y_std,id_0_y_min,id_0_y_max,id_0_z_mean,id_0_z_std,id_0_z_min,id_0_z_max,id_0_visibility_mean,id_0_visibility_std,id_0_visibility_min,id_0_visibility_max,id_1_x_mean,id_1_x_std,id_1_x_min,id_1_x_max,id_1_y_mean,id_1_y_std,id_1_y_min,id_1_y_max,id_1_z_mean,id_1_z_std,id_1_z_min,id_1_z_max,id_1_visibility_mean,id_1_visibility_std,id_1_visibility_min,id_1_visibility_max,id_2_x_mean,id_2_x_std,id_2_x_min,id_2_x_max,id_2_y_mean,id_2_y_std,id_2_y_min,id_2_y_max,id_2_z_mean,id_2_z_std,id_2_z_min,id_2_z_max,id_2_visibility_mean,id_2_visibility_std,id_2_visibility_min,id_2_visibility_max,id_3_x_mean,id_3_x_std,id_3_x_min,id_3_x_max,id_3_y_mean,id_3_y_std,id_3_y_min,id_3_y_max,id_3_z_mean,id_3_z_std,id_3_z_min,id_3_z_max,id_3_visibility_mean,id_3_visibility_std,id_3_visibility_min,id_3_visibility_max,id_4_x_mean,id_4_x_std,id_4_x_min,id_4_x_max,id_4_y_mean,id_4_y_std,id_4_y_min,id_4_y_max,id_4_z_mean,id_4_z_std,id_4_z_min,id_4_z_max,id_4_visibility_mean,id_4_visibility_std,id_4_visibility_min,id_4_visibility_max,id_5_x_mean,id_5_x_std,id_5_x_min,id_5_x_max,id_5_y_mean,id_5_y_std,id_5_y_min,id_5_y_max,id_5_z_mean,id_5_z_std,id_5_z_min,id_5_z_max,id_5_visibility_mean,id_5_visibility_std,id_5_visibility_min,id_5_visibility_max,id_6_x_mean,id_6_x_std,id_6_x_min,id_6_x_max,id_6_y_mean,id_6_y_std,id_6_y_min,id_6_y_max,id_6_z_mean,id_6_z_std,id_6_z_min,id_6_z_max,id_6_visibility_mean,id_6_visibility_std,id_6_visibility_min,id_6_visibility_max,id_7_x_mean,id_7_x_std,id_7_x_min,id_7_x_max,id_7_y_mean,id_7_y_std,id_7_y_min,id_7_y_max,id_7_z_mean,id_7_z_std,id_7_z_min,id_7_z_max,id_7_visibility_mean,id_7_visibility_std,id_7_visibility_min,id_7_visibility_max,id_8_x_mean,id_8_x_std,id_8_x_min,id_8_x_max,id_8_y_mean,id_8_y_std,id_8_y_min,id_8_y_max,id_8_z_mean,id_8_z_std,id_8_z_min,id_8_z_max,id_8_visibility_mean,id_8_visibility_std,id_8_visibility_min,id_8_visibility_max,id_9_x_mean,id_9_x_std,id_9_x_min,id_9_x_max,id_9_y_mean,id_9_y_std,id_9_y_min,id_9_y_max,id_9_z_mean,id_9_z_std,id_9_z_min,id_9_z_max,id_9_visibility_mean,id_9_visibility_std,id_9_visibility_min,id_9_visibility_max,id_10_x_mean,id_10_x_std,id_10_x_min,id_10_x_max,id_10_y_mean,id_10_y_std,id_10_y_min,id_10_y_max,id_10_z_mean,id_10_z_std,id_10_z_min,id_10_z_max,id_10_visibility_mean,id_10_visibility_std,id_10_visibility_min,id_10_visibility_max,id_11_x_mean,id_11_x_std,id_11_x_min,id_11_x_max,id_11_y_mean,id_11_y_std,id_11_y_min,id_11_y_max,id_11_z_mean,id_11_z_std,id_11_z_min,id_11_z_max,id_11_visibility_mean,id_11_visibility_std,id_11_visibility_min,id_11_visibility_max,id_12_x_mean,id_12_x_std,id_12_x_min,id_12_x_max,id_12_y_mean,id_12_y_std,id_12_y_min,id_12_y_max,id_12_z_mean,id_12_z_std,id_12_z_min,id_12_z_max,id_12_visibility_mean,id_12_visibility_std,id_12_visibility_min,id_12_visibility_max,id_13_x_mean,id_13_x_std,id_13_x_min,id_13_x_max,id_13_y_mean,id_13_y_std,id_13_y_min,id_13_y_max,id_13_z_mean,id_13_z_std,id_13_z_min,id_13_z_max,id_13_visibility_mean,id_13_visibility_std,id_13_visibility_min,id_13_visibility_max,id_14_x_mean,id_14_x_std,id_14_x_min,id_14_x_max,id_14_y_mean,id_14_y_std,id_14_y_min,id_14_y_max,id_14_z_mean,id_14_z_std,id_14_z_min,id_14_z_max,id_14_visibility_mean,id_14_visibility_std,id_14_visibility_min,id_14_visibility_max,id_15_x_mean,id_15_x_std,id_15_x_min,id_15_x_max,id_15_y_mean,id_15_y_std,id_15_y_min,id_15_y_max,id_15_z_mean,id_15_z_std,id_15_z_min,id_15_z_max,id_15_visibility_mean,id_15_visibility_std,id_15_visibility_min,id_15_visibility_max,id_16_x_mean,id_16_x_std,id_16_x_min,id_16_x_max,id_16_y_mean,id_16_y_std,id_16_y_min,id_16_y_max,id_16_z_mean,id_16_z_std,id_16_z_min,id_16_z_max,id_16_visibility_mean,id_16_visibility_std,id_16_visibility_min,id_16_visibility_max,id_17_x_mean,id_17_x_std,id_17_x_min,id_17_x_max,id_17_y_mean,id_17_y_std,id_17_y_min,id_17_y_max,id_17_z_mean,id_17_z_std,id_17_z_min,id_17_z_max,id_1

### EDA

Procurando por valores nulos, outliers e outros problemas.

In [18]:
# Análise Exploratória dos Dados

# Verificando as dimensões do dataset
print(f"Dimensões do dataset: {resultados.shape}")

# Verificando os tipos de dados
print("\nTipos de dados:")
print(resultados.dtypes)

# Verificando valores nulos
print("\nValores nulos por coluna:")
print(resultados.isnull().sum())

# Estatísticas descritivas para colunas numéricas
print("\nEstatísticas descritivas para colunas numéricas:")
print(resultados.describe())

# Verificando a distribuição dos sinais (classes)
print("\nDistribuição dos sinais (classes):")
print(resultados['sinal'].value_counts())

# Verificando a distribuição dos intérpretes
print("\nDistribuição dos intérpretes:")
print(resultados['interprete'].value_counts())

# Identificando possíveis outliers em colunas numéricas
print("\nIdentificando outliers em algumas colunas numéricas:")
colunas_numericas = resultados.select_dtypes(include=['int64', 'float64']).columns 

for coluna in colunas_numericas:
    Q1 = resultados[coluna].quantile(0.25)
    Q3 = resultados[coluna].quantile(0.75)
    IQR = Q3 - Q1
    limite_inferior = Q1 - 1.5 * IQR
    limite_superior = Q3 + 1.5 * IQR
    outliers = resultados[(resultados[coluna] < limite_inferior) | (resultados[coluna] > limite_superior)]
    print(f"\nOutliers na coluna {coluna}: {len(outliers)} ({len(outliers)/len(resultados)*100:.2f}%)")

# Criando uma cópia do dataframe para trabalhar com a classificação
df = resultados.copy()

# Criando uma coluna target numérica a partir da coluna 'sinal'
le = LabelEncoder()
df['target'] = le.fit_transform(df['sinal'])

# Mapeamento das classes
print("\nMapeamento das classes (sinal -> target):")
for i, classe in enumerate(le.classes_):
    print(f"{classe} -> {i}")


Dimensões do dataset: (2501, 407)

Tipos de dados:
file_name                 object
width                      int64
height                     int64
duration_sec             float64
num_frames                 int64
sinal                     object
interprete                object
id_0_x_mean              float64
id_0_x_std               float64
id_0_x_min               float64
id_0_x_max               float64
id_0_y_mean              float64
id_0_y_std               float64
id_0_y_min               float64
id_0_y_max               float64
id_0_z_mean              float64
id_0_z_std               float64
id_0_z_min               float64
id_0_z_max               float64
id_0_visibility_mean     float64
id_0_visibility_std      float64
id_0_visibility_min      float64
id_0_visibility_max      float64
id_1_x_mean              float64
id_1_x_std               float64
id_1_x_min               float64
id_1_x_max               float64
id_1_y_mean              float64
id_1_y_std               

### Retirando elementos com baixa visibilidade

In [19]:
# Definindo um limiar para considerar a visibilidade como baixa
limiar_visibilidade = 0.5

# Criando uma lista para armazenar as linhas com visibilidade baixa
linhas_visibilidade_baixa = []

# Identificando todas as colunas relacionadas à visibilidade
colunas_visibilidade = [col for col in resultados.columns if 'visibility' in col]

# Iterando sobre cada linha do DataFrame
for idx, row in resultados.iterrows():
    # Verificando se alguma coluna de visibilidade média está abaixo do limiar
    colunas_visibilidade_media = [col for col in colunas_visibilidade if 'visibility_mean' in col]
    
    # Se alguma coluna de visibilidade média estiver abaixo do limiar, adiciona à lista
    if any(row[col] < limiar_visibilidade for col in colunas_visibilidade_media):
        linhas_visibilidade_baixa.append(idx)

# Exibindo as linhas com visibilidade baixa
print(f"Foram encontradas {len(linhas_visibilidade_baixa)} linhas com visibilidade média abaixo de {limiar_visibilidade}")

# Selecionando apenas as colunas de visibilidade e informações básicas
colunas_para_exibir = ['file_name', 'sinal', 'interprete'] + colunas_visibilidade

# Exibindo todas as linhas com visibilidade baixa, mas apenas as colunas relacionadas à visibilidade
print("\nLinhas com visibilidade baixa (apenas colunas de visibilidade):")

# Verificando quais sinais têm mais problemas de visibilidade
sinais_problematicos = resultados.iloc[linhas_visibilidade_baixa]['sinal'].value_counts()


# Removendo as linhas com visibilidade baixa do DataFrame
resultados = resultados.drop(linhas_visibilidade_baixa)
print(f"\nDataFrame após remover linhas com visibilidade baixa: {resultados.shape}")


Foram encontradas 13 linhas com visibilidade média abaixo de 0.5

Linhas com visibilidade baixa (apenas colunas de visibilidade):

DataFrame após remover linhas com visibilidade baixa: (2488, 407)


### Remoção de colunas

Vou remover as colunas que nao tem relacao com a classificação

In [20]:
# Identificando todas as colunas relacionadas à visibilidade
colunas_visibilidade = [col for col in resultados.columns if 'visibility' in col]

# Exibindo as colunas que serão removidas
print(f"Removendo {len(colunas_visibilidade)} colunas relacionadas à visibilidade:")
print(colunas_visibilidade)

# Removendo as colunas de visibilidade do DataFrame
resultados = resultados.drop(columns=colunas_visibilidade)
print(f"\nDimensões do DataFrame após remover colunas de visibilidade: {resultados.shape}")

# Removendo colunas que não são relevantes para a classificação
colunas_para_remover = ['file_name', 'duration_sec', 'num_frames', 'width', 'height']
resultados = resultados.drop(columns=colunas_para_remover)
print(f"Dimensões do DataFrame após remover colunas irrelevantes: {resultados.shape}")


Removendo 100 colunas relacionadas à visibilidade:
['id_0_visibility_mean', 'id_0_visibility_std', 'id_0_visibility_min', 'id_0_visibility_max', 'id_1_visibility_mean', 'id_1_visibility_std', 'id_1_visibility_min', 'id_1_visibility_max', 'id_2_visibility_mean', 'id_2_visibility_std', 'id_2_visibility_min', 'id_2_visibility_max', 'id_3_visibility_mean', 'id_3_visibility_std', 'id_3_visibility_min', 'id_3_visibility_max', 'id_4_visibility_mean', 'id_4_visibility_std', 'id_4_visibility_min', 'id_4_visibility_max', 'id_5_visibility_mean', 'id_5_visibility_std', 'id_5_visibility_min', 'id_5_visibility_max', 'id_6_visibility_mean', 'id_6_visibility_std', 'id_6_visibility_min', 'id_6_visibility_max', 'id_7_visibility_mean', 'id_7_visibility_std', 'id_7_visibility_min', 'id_7_visibility_max', 'id_8_visibility_mean', 'id_8_visibility_std', 'id_8_visibility_min', 'id_8_visibility_max', 'id_9_visibility_mean', 'id_9_visibility_std', 'id_9_visibility_min', 'id_9_visibility_max', 'id_10_visibility_

In [34]:
resultados.head()

,sinal,interprete,id_0_x_mean,id_0_x_std,id_0_x_min,id_0_x_max,id_0_y_mean,id_0_y_std,id_0_y_min,id_0_y_max,id_0_z_mean,id_0_z_std,id_0_z_min,id_0_z_max,id_1_x_mean,id_1_x_std,id_1_x_min,id_1_x_max,id_1_y_mean,id_1_y_std,id_1_y_min,id_1_y_max,id_1_z_mean,id_1_z_std,id_1_z_min,id_1_z_max,id_2_x_mean,id_2_x_std,id_2_x_min,id_2_x_max,id_2_y_mean,id_2_y_std,id_2_y_min,id_2_y_max,id_2_z_mean,id_2_z_std,id_2_z_min,id_2_z_max,id_3_x_mean,id_3_x_std,id_3_x_min,id_3_x_max,id_3_y_mean,id_3_y_std,id_3_y_min,id_3_y_max,id_3_z_mean,id_3_z_std,id_3_z_min,id_3_z_max,id_4_x_mean,id_4_x_std,id_4_x_min,id_4_x_max,id_4_y_mean,id_4_y_std,id_4_y_min,id_4_y_max,id_4_z_mean,id_4_z_std,id_4_z_min,id_4_z_max,id_5_x_mean,id_5_x_std,id_5_x_min,id_5_x_max,id_5_y_mean,id_5_y_std,id_5_y_min,id_5_y_max,id_5_z_mean,id_5_z_std,id_5_z_min,id_5_z_max,id_6_x_mean,id_6_x_std,id_6_x_min,id_6_x_max,id_6_y_mean,id_6_y_std,id_6_y_min,id_6_y_max,id_6_z_mean,id_6_z_std,id_6_z_min,id_6_z_max,id_7_x_mean,id_7_x_std,id_7_x_min,id_7_x_max,id_7_y_mean,id_7_y_std,id_7_y_min,id_7_y_max,id_7_z_mean,id_7_z_std,id_7_z_min,id_7_z_max,id_8_x_mean,id_8_x_std,id_8_x_min,id_8_x_max,id_8_y_mean,id_8_y_std,id_8_y_min,id_8_y_max,id_8_z_mean,id_8_z_std,id_8_z_min,id_8_z_max,id_9_x_mean,id_9_x_std,id_9_x_min,id_9_x_max,id_9_y_mean,id_9_y_std,id_9_y_min,id_9_y_max,id_9_z_mean,id_9_z_std,id_9_z_min,id_9_z_max,id_10_x_mean,id_10_x_std,id_10_x_min,id_10_x_max,id_10_y_mean,id_10_y_std,id_10_y_min,id_10_y_max,id_10_z_mean,id_10_z_std,id_10_z_min,id_10_z_max,id_11_x_mean,id_11_x_std,id_11_x_min,id_11_x_max,id_11_y_mean,id_11_y_std,id_11_y_min,id_11_y_max,id_11_z_mean,id_11_z_std,id_11_z_min,id_11_z_max,id_12_x_mean,id_12_x_std,id_12_x_min,id_12_x_max,id_12_y_mean,id_12_y_std,id_12_y_min,id_12_y_max,id_12_z_mean,id_12_z_std,id_12_z_min,id_12_z_max,id_13_x_mean,id_13_x_std,id_13_x_min,id_13_x_max,id_13_y_mean,id_13_y_std,id_13_y_min,id_13_y_max,id_13_z_mean,id_13_z_std,id_13_z_min,id_13_z_max,id_14_x_mean,id_14_x_std,id_14_x_min,id_14_x_max,id_14_y_mean,id_14_y_std,id_14_y_min,id_14_y_max,id_14_z_mean,id_14_z_std,id_14_z_min,id_14_z_max,id_15_x_mean,id_15_x_std,id_15_x_min,id_15_x_max,id_15_y_mean,id_15_y_std,id_15_y_min,id_15_y_max,id_15_z_mean,id_15_z_std,id_15_z_min,id_15_z_max,id_16_x_mean,id_16_x_std,id_16_x_min,id_16_x_max,id_16_y_mean,id_16_y_std,id_16_y_min,id_16_y_max,id_16_z_mean,id_16_z_std,id_16_z_min,id_16_z_max,id_17_x_mean,id_17_x_std,id_17_x_min,id_17_x_max,id_17_y_mean,id_17_y_std,id_17_y_min,id_17_y_max,id_17_z_mean,id_17_z_std,id_17_z_min,id_17_z_max,id_18_x_mean,id_18_x_std,id_18_x_min,id_18_x_max,id_18_y_mean,id_18_y_std,id_18_y_min,id_18_y_max,id_18_z_mean,id_18_z_std,id_18_z_min,id_18_z_max,id_19_x_mean,id_19_x_std,id_19_x_min,id_19_x_max,id_19_y_mean,id_19_y_std,id_19_y_min,id_19_y_max,id_19_z_mean,id_19_z_std,id_19_z_min,id_19_z_max,id_20_x_mean,id_20_x_std,id_20_x_min,id_20_x_max,id_20_y_mean,id_20_y_std,id_20_y_min,id_20_y_max,id_20_z_mean,id_20_z_std,id_20_z_min,id_20_z_max,id_21_x_mean,id_21_x_std,id_21_x_min,id_21_x_max,id_21_y_mean,id_21_y_std,id_21_y_min,id_21_y_max,id_21_z_mean,id_21_z_std,id_21_z_min,id_21_z_max,id_22_x_mean,id_22_x_std,id_22_x_min,id_22_x_max,id_22_y_mean,id_22_y_std,id_22_y_min,id_22_y_max,id_22_z_mean,id_22_z_std,id_22_z_min,id_22_z_max,id_23_x_mean,id_23_x_std,id_23_x_min,id_23_x_max,id_23_y_mean,id_23_y_std,id_23_y_min,id_23_y_max,id_23_z_mean,id_23_z_std,id_23_z_min,id_23_z_max,id_24_x_mean,id_24_x_std,id_24_x_min,id_24_x_max,id_24_y_mean,id_24_y_std,id_24_y_min,id_24_y_max,id_24_z_mean,id_24_z_std,id_24_z_min,id_24_z_max
0,0,Alexson,419.308824,5.288022,410.0,428.0,190.963235,8.858901,181.0,207.0,-1.185000,0.127790,-1.41,-0.94,444.764706,6.697966,435.0,455.0,160.588235,7.116460,153.0,173.0,-1.116324,0.117661,-1.34,-0.89,458.522059,6.054037,450.0,467.0,161.698529,7.374952,153.0,175.0,-1.116471,0.117867,-1.34,-0.89,469.551471,4.661616,463.0,477.0,163.029412,7.374572,154.0,177.0,-1.116250,0.117914,-1.34,-0.89,403.022059,6.082722,394.0,412.0,159

In [22]:
backup_resultados = resultados.copy()

In [31]:
resultados = backup_resultados.copy()

### Separar features e target
### Separando train e test sets

In [33]:
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
import pandas as pd

# Definindo coluna alvo
TARGET_COL = 'sinal'

# Codificando o target como inteiro e substituindo no DataFrame
label_encoder = LabelEncoder()
resultados[TARGET_COL] = label_encoder.fit_transform(resultados[TARGET_COL])

# Agora sinal já é inteiro no DataFrame
y = resultados[TARGET_COL].copy()

# Features (removendo target e interprete)
feature_cols = [c for c in resultados.columns if c not in [TARGET_COL, 'interprete']]
X = resultados[feature_cols].copy()

# Definindo intérpretes que vão para o conjunto de teste
test_interpretes = ['Cecilia', 'Everton']

# Criando máscara de teste/treino
mask_test = resultados['interprete'].isin(test_interpretes)

X_train = X[~mask_test].copy()
y_train = y[~mask_test].copy()

X_test = X[mask_test].copy()
y_test = y[mask_test].copy()

print(f"Conjunto de treino: X_train={X_train.shape}, y_train={y_train.shape}")
print(f"Conjunto de teste: X_test={X_test.shape}, y_test={y_test.shape}")

# Criando preprocessador para features numéricas
numeric_features = X_train.select_dtypes(include=['int64', 'float64']).columns.tolist()

preprocess = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
    ],
    remainder='drop'
)

# Distribuição das classes
print("\nDistribuição das classes nos conjuntos:")
print("Treino:", pd.Series(y_train).value_counts().sort_index())
print("Teste:", pd.Series(y_test).value_counts().sort_index())


Conjunto de treino: X_train=(1988, 300), y_train=(1988,)
Conjunto de teste: X_test=(500, 300), y_test=(500,)

Distribuição das classes nos conjuntos:
Treino: sinal
0     80
1     81
2     80
3     80
4     80
5     80
6     80
7     80
8     80
9     80
10    80
11    80
12    80
13    80
14    80
15    80
16    80
17    80
18    80
19    80
20    70
21    80
22    77
23    80
24    80
Name: count, dtype: int64
Teste: sinal
0     21
1     20
2     20
3     20
4     20
5     20
6     19
7     20
8     20
9     20
10    20
11    20
12    20
13    20
14    20
15    20
16    20
17    20
18    20
19    20
20    20
21    20
22    20
23    20
24    20
Name: count, dtype: int64


### Implementando Random Forest

- Adição: 0
- Aluno: 1
- Antropologia: 2
- Apontador: 3
- Apostila: 4
- Biologia: 5
- Bolsa de Estudos: 6
- Capítulo: 7
- Classe: 8
- Coerência: 9
- Coesão: 10
- Colega: 11
- Conceito: 12
- Contexto: 13
- Curso: 14
- Dicionário: 15
- Disciplina: 16
- Ensinar: 17
- Escola: 18
- Estudar: 19
- Filosofia: 20
- Física: 21
- Geografia: 22
- História: 23
- Ângulo: 24

In [36]:


# Usar os conjuntos de treino já definidos anteriormente
# Criar pipeline: escalonamento + modelo
pipeline = Pipeline([
    ("scaler", StandardScaler()),   # pode ser ignorado no RF, mas útil se trocar para KNN/MLP
    ("rf", RandomForestClassifier(random_state=42, n_jobs=-1))
])

# Definir o k-fold estratificado apenas para os dados de treino
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# F1-score macro
f1_macro = make_scorer(f1_score, average="macro")

# Avaliar com validação cruzada apenas nos dados de treino
scores = cross_val_score(pipeline, X_train, y_train, cv=skf, scoring=f1_macro, n_jobs=-1)

print("F1-scores por fold:", scores)
print("Média do F1-score na validação cruzada:", np.mean(scores))

# Treinar o modelo final com todos os dados de treino
pipeline.fit(X_train, y_train)

# Avaliar no conjunto de teste separado
y_pred = pipeline.predict(X_test)
test_score = f1_score(y_test, y_pred, average="macro")
print(f"F1-score no conjunto de teste: {test_score:.4f}")

F1-scores por fold: [0.98499022 0.97742945 0.9722868  0.98489932 0.98991447]
Média do F1-score na validação cruzada: 0.981904051485652
F1-score no conjunto de teste: 0.1096


In [38]:
print("\n--- Targets ---")
print("y_train:", y_train.shape, y_train.dtype)
print("y_test:", y_test.shape, y_test.dtype)

print("\n--- Features Treino ---")
X_train.info()

print("\n--- Features Teste ---")
X_test.info()


--- Targets ---
y_train: (1988,) int64
y_test: (500,) int64

--- Features Treino ---
<class 'pandas.core.frame.DataFrame'>
Index: 1988 entries, 0 to 2500
Columns: 300 entries, id_0_x_mean to id_24_z_max
dtypes: float64(300)
memory usage: 4.6 MB

--- Features Teste ---
<class 'pandas.core.frame.DataFrame'>
Index: 500 entries, 501 to 1500
Columns: 300 entries, id_0_x_mean to id_24_z_max
dtypes: float64(300)
memory usage: 1.1 MB


### Implementando KNN

## Modelagem e Avaliação (CV Estratificada)

Treinaremos e avaliaremos:
- Random Forest
- KNN
- MLP

Usaremos F1 macro e exibiremos a matriz de confusão a partir de `cross_val_predict` para cada modelo.


In [ ]:
def avaliar_modelo(nome, estimator):
    pipe = Pipeline(steps=[('preprocess', preprocess), ('model', estimator)])
    # cross_validate para f1 macro
    cv_results = cross_validate(
        pipe, X, y, cv=cv, scoring=scorer, n_jobs=N_JOBS, return_estimator=False, error_score='raise'
    )

    # cross_val_predict para matriz de confusão
    y_pred = cross_val_predict(pipe, X, y, cv=cv, n_jobs=N_JOBS, method='predict')

    f1_mean = np.mean(cv_results['test_f1_macro'])
    f1_std = np.std(cv_results['test_f1_macro'])

    print(f"\nModelo: {nome}")
    print(f"F1-macro (média ± std): {f1_mean:.4f} ± {f1_std:.4f}")

    cm = confusion_matrix(y, y_pred)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm)
    disp.plot(cmap='Blues', values_format='d')
    plt.title(f'Matriz de Confusão - {nome}')
    plt.show()

# Modelos com hiperparâmetros básicos e random_state fixo
rf = RandomForestClassifier(n_estimators=300, random_state=RANDOM_STATE, n_jobs=N_JOBS)
knn = KNeighborsClassifier(n_neighbors=5, weights='distance')
mlp = MLPClassifier(hidden_layer_sizes=(100,), max_iter=500, random_state=RANDOM_STATE)

for nome, est in [
    ('Random Forest', rf),
    ('KNN', knn),
    ('MLP', mlp)
]:
    avaliar_modelo(nome, est)
